In [1]:
#Evaluating the model means finding out the testing accuracy of model
#and saving the accuracy score in json format

#step3:-updating the entity file...entity file is nothing but whatever parameter we have define in yaml file
#we used as a class variable init
from src.Mask.exceptions import CustomException
import os,sys
from src.Mask.loggers import logger
from src.Mask.Constants import *
from src.Mask.Utils import read_yaml,create_directory,save_json
from dataclasses import dataclass
from pathlib import Path

In [2]:
@dataclass
class EvaluatingModelConfig():
    root_dirpath:Path
    model_trained_dirpath:Path
    all_param:dict
    data_dirpath:Path
    score_dirpath:Path


In [3]:
#step4)updating the configurationmanager file :
#in this file we read the yaml file and create directories accordingly then 
#getting the dirpath from yaml file as value and assigning them into class variable and taking rtn as function
class ConfigurationManager():
    def __init__(self,config_filepath=CONFIG_FILEPATH,param_filepath=PARAM_FILEPATH):
        self.config = read_yaml(config_filepath) #rtn the yaml file as configbox dictatonary
        self.param = read_yaml(param_filepath)   ##rtn the yaml file as configbox dictatonary

        #creating artifacts directory into project structure
        create_directory([self.config.artifact_root_dir]) #will create artifacts directory

    def get_evaluation_config(self) ->EvaluatingModelConfig:
        #initalizing the local variable which is used by this method only
        config = self.config.evaluating_model
        param = self.param

        #creating directory artifacts/model_evaluation 
        create_directory([config.root_dirpath])

        #creating an object of EvaluatingModelConfig class and assigning the value to it
        model_evaluation_config = EvaluatingModelConfig(
            root_dirpath=config.root_dirpath,
            model_trained_dirpath= config.model_trained_dirpath,
            all_param=param,
            data_dirpath=config.data_dirpath,
            score_dirpath=config.score_dirpath
        )

        return model_evaluation_config

In [4]:
import tensorflow as tf

In [5]:
#step5)updating the component files : 
#in this file we create and object for class variable and perform task accordingly!!!
class EvaluatingModel():
    def __init__(self,evaluateconfig:EvaluatingModelConfig):
        self.evaluateconfig = evaluateconfig

    #Now loading trained model to evaluating the testing accuracy of it
    def load_trained_model(self):
        self.trained_model  = tf.keras.models.load_model(self.evaluateconfig.model_trained_dirpath)
        
    #creating method to perform evaluation on testing data so we r
    # using prepreocessing techniques of image data generator of tensorflow 

    def _test_generator(self):
        #creating keyward argment for datagenerator
        datagenerator_kwarg = dict(
            rescale=1./255,
            validation_split=0.20
        )

        #creating keyward argment for dataflow
        dataflow_kwarg = dict(
            target_size=self.evaluateconfig.all_param.input_shape[:-1],
            batch_size=self.evaluateconfig.all_param.batch_size,
            interpolation = "bilinear"
        )

        #now generating the valid data for testing purpose!!!
        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwarg
        )

        #now this valid_data_generator we are flowing from directory
        self.valid_data_generator = valid_data_generator.flow_from_directory(
            directory=self.evaluateconfig.data_dirpath,
            shuffle=True,
            subset='validation',
            **dataflow_kwarg
        )

    

    def evaluating(self):
        #now we are creating the model object and compiling it!!!
        
        self.load_trained_model()  #now loading the method of same class

        evaluating_score = self.trained_model.evaluate(self.valid_data_generator)

        save_json(
            file = Path(self.evaluateconfig.score_dirpath),
            data = {
                "loss":evaluating_score[0],
                "accuracy": evaluating_score[1]
            }
        )

        print(evaluating_score)

In [6]:
os.chdir('../')
%pwd

'd:\\MaskImageDetection\\MaskDetectionModel'

In [7]:
#step6) updating the training pipeline
try:

    cm = ConfigurationManager()
    evaluate_config = cm.get_evaluation_config()

    em = EvaluatingModel(evaluateconfig=evaluate_config)

    em.load_trained_model()

    em._test_generator()

    em.evaluating()

except Exception as e:
    raise CustomException(e,sys)

[2024-10-03 21:51:52,785]-INFO-33-Yaml file config\config.yaml reading
[2024-10-03 21:51:52,796]-INFO-33-Yaml file param.yaml reading
Found 12 images belonging to 2 classes.
2/2 [==============================] - 3s 522ms/step - loss: 0.6504 - accuracy: 0.6667
[2024-10-03 21:51:57,844]-INFO-79-Json file saved at artifacts\model_evaluation\score.json
[0.6504024863243103, 0.6666666865348816]
